# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

jeopardy = pd.read_csv('jeopardy.csv')

# investigating column names to learn original in order to rename to more python friendly version
print(jeopardy.columns)

# renaming the columns to make them easier to call and fixing the columns Question and Answer as Jeopardy!'s format is that the question/clue that is provided is the "answer" and the response the contestant's give is the "question"
jeopardy.rename(columns = {'Show Number': 'show_number', ' Air Date': 'air_date', ' Round': 'round', ' Category': 'category', ' Value': 'value', ' Question': 'answer', ' Answer': 'question'}, inplace = True)

# verifying column rename and investigating the format of the data
print(jeopardy.head())

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')
   show_number    air_date      round                         category value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY  $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES  $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...  $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE  $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES  $200   

                                                                                                        answer  \
0             For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory   
1  No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves   
2                     The city of Yuma in this state ha

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [2]:
# Initially attempted to use .isin and .str.contains without success. Found this article, https://stackoverflow.com/questions/26577516/how-to-test-if-a-string-contains-one-of-the-substrings-in-a-list-in-pandas, 
# and adapted the lambda function solution at the bottom to fit my needs


searchfor = ['King', 'England']
def search_answers(dataset, searchfor):
    temporary_datase = dataset
    temporary_datase["TrueFalse"] = temporary_datase['answer'].apply(lambda x: 1 if all(i in x for i in searchfor) else 0)
    return temporary_datase.answer[temporary_datase["TrueFalse"] == 1]


print(len(search_answers(jeopardy, searchfor)))
print(search_answers(jeopardy, searchfor).head())

49
4953                 Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
14912       This country's King Louis IV was nicknamed "Louis From Overseas" because he was raised in England
21511                            this man and his son ruled England following the execution of King Charles I
23810    William the Conqueror was crowned King of England in Westminster Abbey on Christmas Day in this year
27555                      This member of the Medici family was the mother-in-law of England's King Charles I
Name: answer, dtype: object


3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [3]:
# enter lowercase words only into the search for list

searchfor = ['king', 'england']
def search_answers(dataset, searchfor):
    temporary_datase = dataset
    temporary_datase['answer'] = temporary_datase['answer'].str.lower()
    temporary_datase["TrueFalse"] = temporary_datase['answer'].apply(lambda x: 1 if all(i in x for i in searchfor) else 0)
    return temporary_datase.answer[temporary_datase["TrueFalse"] == 1]


print(len(search_answers(jeopardy, searchfor)))
print(search_answers(jeopardy, searchfor).head())

152
4953                   both england's king george v & fdr put their stamp of approval on this "king of hobbies"
6337     in retaliation for viking raids, this "unready" king of england attacks norse areas of the isle of man
9191                   this king of england beat the odds to trounce the french in the 1415 battle of agincourt
11710              this scotsman, the first stuart king of england, was called "the wisest fool in christendom"
13454                                      it's the number that followed the last king of england named william
Name: answer, dtype: object


4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [4]:
jeopardy['value_float'] = jeopardy["value"].str.replace("$","", regex=True)
jeopardy['value_float'] = jeopardy["value_float"].str.replace("None","0", regex=True)
jeopardy['value_float'] = jeopardy["value_float"].str.replace(",","", regex=True)

jeopardy['value_float'] = jeopardy['value_float'].astype(float, errors = 'raise')

#print(jeopardy.value.head())
#print(jeopardy.value_float.head())
print(jeopardy.value_float.head())

print(jeopardy.value_float.mean())
print(jeopardy.value_float[jeopardy["TrueFalse"] == 1].mean())

0    200.0
1    200.0
2    200.0
3    200.0
4    200.0
Name: value_float, dtype: float64
739.9884755451067
886.8421052631579


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [5]:
# only search lowercase terms in the unique_questions variable

unique_questions = ['king']
def search_unique(dataset, unique_questions):
    dataset['answer'] = dataset['answer'].str.lower()
    dataset["TrueFalse2"] = dataset['answer'].apply(lambda x: 1 if all(i in x for i in unique_questions) else 0)
    return dataset.question[dataset["TrueFalse2"] == 1].value_counts()

print(search_unique(jeopardy, unique_questions))

Henry VIII                   55
Solomon                      35
Richard III                  33
Louis XIV                    31
David                        30
                             ..
cardiac (in card I acted)     1
Henderson                     1
Computer                      1
Indians                       1
work                          1
Name: question, Length: 5268, dtype: int64


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [6]:
compare_questions = ['king']

jeopardy["year"]= jeopardy.air_date.apply(lambda air_date: air_date.split('-')[0])

def search_unique(dataset, compare_questions):
    dataset['answer'] = dataset['answer'].str.lower()
    dataset["TrueFalse3"] = dataset['answer'].apply(lambda x: 1 if all(i in x for i in compare_questions) else 0)
    return dataset.question[dataset["TrueFalse3"] == 1].count()

print("Number of time " + str(compare_questions) + " appeared in an answer in our dataset")
print(search_unique(jeopardy, compare_questions))

print("Number of time " + str(compare_questions) + " appeared in an answer in our dataset for the year 1984")
      
def search_unique_timeframe(dataset, compare_questions, timeframe):
    dataset['answer'] = dataset['answer'].str.lower()
    dataset['correct_date'] = dataset['year'].apply(lambda y: 1 if y in timeframe else 0)
    return dataset.year[(dataset["correct_date"] == 1) & (dataset['TrueFalse3'] == 1)].count()


question_1984 = search_unique_timeframe(jeopardy, compare_questions, '1984')
print(question_1984)

print("Number of time " + str(compare_questions) + " appeared in an answer in our dataset for the year 1994")
question_1994 = search_unique_timeframe(jeopardy, compare_questions, '1994')
print(question_1994)

print("Number of time " + str(compare_questions) + " appeared in an answer in our dataset for the year 2004")
question_2004 = search_unique_timeframe(jeopardy, compare_questions, '2004')
print(question_2004)

# 2011 is the latest year in the dataset. Found by investigating with print(jeopardy.year.unique())

print("Number of time " + str(compare_questions) + " appeared in an answer in our dataset for the year 2014")
question_2011 = search_unique_timeframe(jeopardy, compare_questions, '2011')
print(question_2011)


Number of time ['king'] appeared in an answer in our dataset
7409
Number of time ['king'] appeared in an answer in our dataset for the year 1984
19
Number of time ['king'] appeared in an answer in our dataset for the year 1994
42
Number of time ['king'] appeared in an answer in our dataset for the year 2004
431
Number of time ['king'] appeared in an answer in our dataset for the year 2014
448


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.

In [7]:
print(jeopardy.year.unique())

['2004' '2010' '2000' '2006' '2009' '1996' '2007' '1997' '2002' '2003'
 '2001' '1990' '1999' '2008' '2011' '2005' '1998' '2012' '1992' '1986'
 '1985' '1991' '1993' '1994' '1988' '1987' '1995' '1984' '1989']
